# Special Flood Inundation Mapping

This notebook demonstrates how to create special FIM maps without using gauge stage. Those maps include:
* Flood depth map using a constant DOF at all the stream pixels/cells
* 'MinDtf' map -- the minimum depth to flood (DTF) needed to inundate a FPP
* 'NumOfFsps' map -- the number of FSPs that may inundate a FPP
* 'Depression' map -- he depression depth filled

## Import Modules

Import necessary modules.

In [1]:
import sys
import time
from dask.distributed import Client, LocalCluster
from dask import visualize

### Import FLDPLN Modules

In [2]:
# Tool/script folder
# fldplnToolFolder = r'D:\xingong\tools' # for making available to the public
fldplnToolFolder = r'E:\CUAHSI_SI\training\source' # tool development folder, has the latest version

# Add the tool/script folder to sys.path to access fldpln modules
sys.path.append(fldplnToolFolder) 
# fldpln module
from fldpln import *
from fldpln_gauge import *

## Setup Tiled Libraries and Output Map Folders

Here we setup the folder under which tiled libraries (organized as folders) are located. We also setup the output folder (i.e., outputFolder) under which a map folder and a 'scratch' folder are created. The map folder, which is specified later, comtains all inundation depth maps. The scratch folder stores temporary files.

In [3]:
# tiled library folder that may have more than one tiled libraries
libFolder =  r'E:\CUAHSI_SI\training\examples\wildcat_10m_3dep\tiled_snz_library'

# Set output folder
outputFolder = r'E:\CUAHSI_SI\training\examples\wildcat_10m_3dep\maps'

## Setup Mapping Parameters

There are several parameters need to be set before geenrating the maps. Those include:
* The FLDPLN libraries to map (i,e,, varibale libs2Map)
* Which special map to create (i.e., specialDof)
* Name of the map folder (i.e., outMapFolderName) which is under the output folder (i.e., outputFolder) and stores all inundation depth maps
* Whether to mosaic tiles as single COG TIFF file and whether to use a Dask local cluster to speed up the mapping

In [4]:
# select the libraries under tiled library folder to generate maps
libs2Map = ['lib_py']

# set the special map wanted
# 'MinDtf','NumOfFsps','Depression', or a real number (for example 20.5, in foot for KS libraries) representing constant DOF at all the FSPs
specialDof = 'NumOfFsps' # Wildcat 3DEP DEM has a vertical unit of meters

# set up output map-folder under the outputFolder
outMapFolderName = 'numoffsps'

# Create folders for storing temp and output map files
outMapFolder,scratchFolder = CreateFolders(outputFolder,'scratch',outMapFolderName)

# whether mosaci tiles as a single COG
mosaicTiles = True #True #False

# Using LocalCluster by default
useLocalCluster = False
numOfWorkers = round(0.8*os.cpu_count())

In [5]:
scratchFolder

'E:\\CUAHSI_SI\\training\\examples\\wildcat_10m_3dep\\maps\\scratch'

## Generate Special Map

In [6]:

# show mapping info
print(f'Tiled FLDPLN library folder: {libFolder}')
print(f'Map folder: {outMapFolder}')
# libs to map
print(f'Libraries to map: {libs2Map}')

# check running time
startTimeAllLibs = time.time()

# create a local cluster to speed up the mapping. Must be run inside "if __name__ == '__main__'"!!!
if useLocalCluster:
    # cluster = LocalCluster(n_workers=4,processes=False)
    try:
        print('Start a LocalCluster ...')
        # NOTE: set worker space (i.e., local_dir) to a folder that the LocalCluster can access. When run the script through a scheduled task, 
        # the system uses C:\Windows\system32 by default, which a typical user doesn't have the access!
        cluster = LocalCluster(n_workers=numOfWorkers,memory_limit='32GB',local_dir="D:/projects_new/fldpln/tools") # for KARS production server (192G RAM & 8 cores)
        # cluster = LocalCluster(n_workers=numOfWorkers,processes=False) # for KARS production server (192G RAM & 8 cores)
        # print('Watch workers at: ',cluster.dashboard_link)
        print(f'Number of workers: {numOfWorkers}')
        client = Client(cluster)
        # print scheduler info
        # print(client.scheduler_info())
    except:
        print('Cannot create a LocalCLuster!')
        useLocalCluster = False

# dict to store lib processing time
libTime={}

# map each library
for libName in libs2Map:
    # check running time
    startTime = time.time()
    
    # mapping flood depth
    if useLocalCluster:
        print(f'Map [{libName}] using LocalCLuster ...')
        # generate a DAG
        dag,dagRoot=MapFloodDepthWithTilesAsDag(libFolder,libName,'snappy',outMapFolder,fspDof=specialDof,aoiExtent=None)
        if dag is None:
            tileTifs = None
        else:
            # visualize DAG
            # visualize(dag)
            # Compute DAG
            tileTifs = client.get(dag, dagRoot)
            if not tileTifs: # list is empty
                tileTifs =  None
    else:
        print(f'Map {libName} ...')
        tileTifs = MapFloodDepthWithTiles(libFolder,libName,'snappy',outMapFolder,fspDof=specialDof,aoiExtent=None)
    print(f'Actual mapped tiles: {tileTifs}')

    # Mosaic all the tiles from a library into one tif file
    if mosaicTiles and not(tileTifs is None):
        print('Mosaic tile maps ...')
        mosaicTifName = libName+'_'+outMapFolderName+'.tif'
        # Simplest implementation, may crash with very large raster
        MosaicGtifs(outMapFolder,tileTifs,mosaicTifName,keepTifs=False)
    
    # check time
    endTime = time.time()
    usedTime = round((endTime-startTime)/60,3)
    libTime[libName] = usedTime
    # print(f'{libName} processing time (minutes):', usedTime)

# Show processing time
# Individual lib processing time
print('Individual library mapping time:', libTime)
# total time
endTimeAllLibs = time.time()
print('Total processing time (minutes):', round((endTimeAllLibs-startTimeAllLibs)/60,3))

#
# Shutdown local clusters
#
if useLocalCluster:
    print('Shutdown LocalCluster ...')
    cluster.close()
    client.shutdown()
    client.close()
    useLocalCluster = False

Tiled FLDPLN library folder: E:\CUAHSI_SI\training\examples\wildcat_10m_3dep\tiled_snz_library
Map folder: E:\CUAHSI_SI\training\examples\wildcat_10m_3dep\maps\numoffsps
Libraries to map: ['lib_py']
Map lib_py ...
Tiles need to be mapped: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]
Actual mapped tiles: ['E:\\CUAHSI_SI\\training\\examples\\wildcat_10m_3dep\\maps\\numoffsps\\lib_py_tile_1.tif', 'E:\\CUAHSI_SI\\training\\examples\\wildcat_10m_3dep\\maps\\numoffsps\\lib_py_tile_2.tif', 'E:\\CUAHSI_SI\\training\\examples\\wildcat_10m_3dep\\maps\\numoffsps\\lib_py_tile_3.tif', 'E:\\CUAHSI_SI\\training\\examples\\wildcat_10m_3dep\\maps\\numoffsps\\lib_py_tile_4.tif', 'E:\\CUAHSI_SI\\training\\examples\\wildcat_10m_3dep\\maps\\numoffsps\\lib_py_tile_5.tif', 'E:\\CUAHSI_SI\\training\\examples\\wildcat_10m_3dep\\maps\\numoffsps\\lib_py_ti